In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.models import load_model
from keras.utils import np_utils
from keras import regularizers
import time


Using TensorFlow backend.


In [2]:
# Import dataset
dataset = pd.read_csv('C:/Users/niharika/Desktop/data sciences/Springboard/Capstone 2/Stock/ABT_RNN.csv', encoding = "utf8", parse_dates=True)
dataset.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Difference High,Label
0,29430,0.638281,0.643542,0.633020,0.640034,0.286761,2309500,NaN,NaN
1,29431,0.645295,0.655816,0.645295,0.655816,0.293831,2224000,0.012274,1.0
2,29432,0.661077,0.683872,0.661077,0.675105,0.302474,4875700,0.028056,1.0
3,29433,0.675105,0.675105,0.663049,0.675105,0.302474,3328800,-0.008767,0.0
4,29434,0.675105,0.687379,0.673351,0.677077,0.303357,5339000,0.012274,1.0


In [3]:
#Split to Data samples and labels
x = dataset.iloc[1:, 0:7].values
y = dataset.iloc[1:, 8].values

In [4]:
x

array([[  2.94310000e+04,   6.45295000e-01,   6.55816000e-01, ...,
          6.55816000e-01,   2.93831000e-01,   2.22400000e+06],
       [  2.94320000e+04,   6.61077000e-01,   6.83872000e-01, ...,
          6.75105000e-01,   3.02474000e-01,   4.87570000e+06],
       [  2.94330000e+04,   6.75105000e-01,   6.75105000e-01, ...,
          6.75105000e-01,   3.02474000e-01,   3.32880000e+06],
       ..., 
       [  4.29400000e+04,   5.08100010e+01,   5.09900020e+01, ...,
          5.07999990e+01,   5.07999990e+01,   8.58430000e+06],
       [  4.29410000e+04,   5.09800000e+01,   5.10000000e+01, ...,
          5.08300020e+01,   5.08300020e+01,   5.69280000e+06],
       [  4.29420000e+04,   5.07400020e+01,   5.08100010e+01, ...,
          5.04000020e+01,   5.04000020e+01,   5.95800000e+06]])

In [5]:
#Number of time series to work on is 1 (today depends on yesterday)
t=1
n_samples= np.floor(len(dataset)/t)

In [6]:
x.shape

(9330, 7)

In [7]:
#Reshape input samples to Nx1x7
x = np.reshape(x, (x.shape[0]//t, t, x.shape[1]))

#Set labels to these N samples
indices=[ind for ind in range(len(y)) if ind%t==0]
y=y[indices]

In [8]:
y

array([ 1.,  1.,  0., ...,  1.,  1.,  0.])

In [9]:
# One-hot encode the classes (1.0 and 0.0 will be considered as float values otherwise)
from sklearn.preprocessing import LabelEncoder
label_encoder=LabelEncoder()
y_new=label_encoder.fit_transform(y)
y_one_hot=np_utils.to_categorical(y_new)
y=y_one_hot

In [10]:
y

array([[ 0.,  1.],
       [ 0.,  1.],
       [ 1.,  0.],
       ..., 
       [ 0.,  1.],
       [ 0.,  1.],
       [ 1.,  0.]])

In [11]:
#Split data to training and testing data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)

In [12]:
#Construct the model
n_neurons=7
start_time=time.time()
model = Sequential()
model.add(LSTM(n_neurons, input_shape=(X_train.shape[1],X_train.shape[2])))
model.add(Dense( activation="tanh", kernel_initializer="uniform", units=64, activity_regularizer=regularizers.l2(0.01)))
model.add(Dense( activation="tanh", kernel_initializer="uniform", units=32, activity_regularizer=regularizers.l2(0.01)))
model.add(Dense( activation="softmax", kernel_initializer="uniform", units=2))
model.compile(loss='mse', optimizer='adam',metrics=['accuracy'])

In [13]:
#Train the model 
model.fit(X_train, y_train, epochs=100, batch_size=100)

Epoch 1/100
7464/7464 [==============================] - 34s - loss: 0.2550 - acc: 0.5263    
Epoch 2/100
7464/7464 [==============================] - 2s - loss: 0.2494 - acc: 0.5263     
Epoch 3/100
7464/7464 [==============================] - 1s - loss: 0.2493 - acc: 0.5263     
Epoch 4/100
7464/7464 [==============================] - 1s - loss: 0.2493 - acc: 0.5263     
Epoch 5/100
7464/7464 [==============================] - 1s - loss: 0.2493 - acc: 0.5263     
Epoch 6/100
7464/7464 [==============================] - 1s - loss: 0.2493 - acc: 0.5263     
Epoch 7/100
7464/7464 [==============================] - 1s - loss: 0.2493 - acc: 0.5263     
Epoch 8/100
7464/7464 [==============================] - 1s - loss: 0.2493 - acc: 0.5263     
Epoch 9/100
7464/7464 [==============================] - 1s - loss: 0.2493 - acc: 0.5263     
Epoch 10/100
7464/7464 [==============================] - 1s - loss: 0.2494 - acc: 0.5263     
Epoch 11/100
7464/7464 [==============================] - 1

In [14]:
#Evaluation of the Training time
print("%s Minutes of Execution" %str((time.time()-start_time)/60))

#Save the model for prediction
model.save('model_test.h5')
print ("Model Saved")

2.5500335097312927 Minutes of Execution
Model Saved


In [15]:
# Evaluate the model on CV Data
scores = model.evaluate(X_test, y_test, verbose=1)
print ("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

1866/1866 [==============================] - 1s     
acc: 53.11%


In [19]:
pred = model.predict(np.array(X_test)) 
pred

array([[ 0.52705836,  0.4729417 ],
       [ 0.52705836,  0.4729417 ],
       [ 0.52705836,  0.4729417 ],
       ..., 
       [ 0.52705836,  0.4729417 ],
       [ 0.52705836,  0.4729417 ],
       [ 0.52705836,  0.4729417 ]], dtype=float32)

yp = model.predict(X_test, batch_size=32, verbose=1)
ypreds = np.argmax(yp, axis=1)
print(average_precision_score(ytrue, ypreds))
print(accuracy_score(ytrue, ypreds))

In [18]:
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score 
from sklearn.metrics import confusion_matrix

print (classification_report(y_test, pred))
print (accuracy_score(y_test, pred))

ValueError: Mix type of y not allowed, got types {'multilabel-indicator', 'continuous-multioutput'}

import pandas as pd

pd.crosstab(y_test, pred, rownames=["Actual"], colnames=["Predicted"])